Name: David L. Parks  
Prepared: Fall 2019

a. What website you scraped, for what kind of products?  
I scraped Trip Advisor for restaurants in Roanoke, Va.

b. How you designed your scraping?  How you selected a list of products on the site?  
I started my crawler on the page for restaurants in Roanoke. I then crawled across 5 pages of restaurants collecting 150 urls in a list. I used that list to fetch the page for each restaurant and scrape the latest reviews, up to 10, for each restaurant.  

c. How you minimized your impact on the targeted website?  
I changed the header to reflect my current web browser and I used time.sleep() with the numpy random int generator to randomly wait for 3 to 5 seconds between page requests.

d. Any challenges you encountered and how you addressed them?  
My initial crawler was getting a null pointer error because some restaurants only had one page of reviews. I fixed this issue by limiting the crawler to the first page of reviews.

In [1]:
import bs4 as bs
import numpy as np
import pandas as pd
import requests
import time

#### Set the starting page.

In [2]:
# Trip Advisor reviews for restaurants in Roanoke, VA
homepage = 'https://www.tripadvisor.com'
url = 'https://www.tripadvisor.com/Restaurants-g58134-Roanoke_Virginia.html'
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0'}

#### Crawl across 5 pages and scrape the url for each restaurant.

In [3]:
restaurant_urls = []

In [4]:
# each iteration of the loop will randomly sleep from [3, 5] seconds
for i in range(5):
    page = requests.get(url, headers=header)
    soup = bs.BeautifulSoup(page.text, 'html5lib')
    restaurants = soup.find_all('a', {'class': 'restaurants-list-ListCell__restaurantName--2aSdo'})
    restaurant_urls = restaurant_urls + [homepage + restaurant.get('href') for restaurant in restaurants]
    url = homepage + soup.find('a', {'class': 'nav next rndBtn ui_button primary taLnk'}).get('href')
    time.sleep(np.random.randint(3, 6))

#### Delete any duplicate urls.

In [5]:
restaurant_urls = list(set(restaurant_urls))

#### Scrape up to 10 reviews from each restaurant.

In [6]:
restaurant_reviews = []

In [7]:
# each iteration of the loop sleeps for 1 second
for restaurant_url in restaurant_urls:
    page = requests.get(restaurant_url, headers=header)
    soup = bs.BeautifulSoup(page.text, 'html5lib')
    
    # get restaurant name
    name = soup.find('h1', {'class': 'ui_header h1'}).text
    
    # get urls of first page of reviews
    reviews = soup.find_all('a', {'class': 'title'})
    review_urls = [homepage + review.get('href') for review in reviews]
    
    # sleep for 1 second
    time.sleep(1)
    
    # each iteration of the loop will randomly sleep from [3, 5] seconds
    for review_url in review_urls:
        page = requests.get(review_url, headers=header)
        soup = bs.BeautifulSoup(page.text, 'html5lib')
        rating = float(str(soup.find('div', {'class': 'rating reviewItemInline'}).contents[0])[-11:-10])
        date = soup.find('span', {'class': 'ratingDate relativeDate'}).get('title')
        author = soup.find('span', {'class': 'expand_inline scrname'}).text
        text = soup.find('p', {'class': 'partial_entry'}).text
        # strips extra space from the ends of the text and removes any non-ascii characters
        text = text.strip().encode('ascii', 'ignore').decode()
        restaurant_reviews.append([name, rating, date, author, text, review_url])
        time.sleep(np.random.randint(3, 6))

#### Convert restaurant review list into a dataframe.

In [8]:
restaurant_reviews = pd.DataFrame(restaurant_reviews,
                                 columns=['Restaurant', 'Rating', 'Date', 'Author', 'Text', 'URL'])

#### Export dataframe to csv.

In [14]:
restaurant_reviews.to_csv('Homework08.csv', index=False)